## SetUp

In [3]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [4]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader, ExperimentConfig}
import com.flightdelay.data.loaders.FlightDataLoader

// Env Configuration
val args: Array[String] = Array("jupyter")
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)
implicit val experimentConfig: ExperimentConfig = configuration.experiments(0)

val spark = SparkSession.builder()
  .config(sc.getConf)
  .config("spark.eventLog.enabled", "true")
  .config("spark.eventLog.dir", s"${configuration.common.output.basePath}/spark-events")  // ex: "file:/tmp/spark-events" ou "hdfs:///spark-events"
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark


args = Array(jupyter)
configuration = AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...


AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...

In [19]:
val joinedDataPath = s"${configuration.common.output.basePath}/Experience-local-base/data/join_exploded_train_prepared.parquet"
val joinedData = spark.read.parquet(joinedDataPath)

println("Joined DF Count: ", joinedData.count())


(Joined DF Count: ,117706)


joinedDataPath = /home/jovyan/work/output/Experience-local-base/data/join_exploded_train_prepared.parquet
joinedData = [CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 170 more fields]


[CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 170 more fields]

In [20]:
joinedData.printSchema

root
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- D1: integer (nullable = true)
 |-- D2_15: integer (nullable = true)
 |-- D2_30: integer (nullable = true)
 |-- D2_45: integer (nullable = true)
 |-- D2_60: integer (nullable = true)
 |-- D2_90: integer (nullable = true)
 |-- D3: integer (nullable = true)
 |-- D4: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- UTC_CRS_DEP_TIME: string (nullable = true)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- depHour: integer (nullable = true)
 |-- feature_arrival_time_period: string (nullable = true)
 |-- feature_departure_hour_rounded: long (nullable 

In [7]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

def countMissing(df: org.apache.spark.sql.DataFrame): org.apache.spark.sql.DataFrame = {

  val exprs = df.schema.fields.map { f =>
    val c  = f.name
    val dt = f.dataType

    val cond = dt match {
      case DoubleType | FloatType =>
        col(c).isNull || isnan(col(c))
      case StringType =>
        col(c).isNull || col(c) === ""
      case _ =>
        col(c).isNull
    }

    sum(when(cond, 1).otherwise(0)).alias(c)
  }

  df.agg(exprs.head, exprs.tail: _*)
}

// Exemple :
countMissing(joinedData).show(200, 1000, true)

countMissing: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


-RECORD 0-------------------------------------------------------
 CRS_ARR_TIME                                           | 0     
 CRS_DEP_TIME                                           | 0     
 CRS_ELAPSED_TIME                                       | 0     
 D1                                                     | 0     
 D2_15                                                  | 0     
 D2_30                                                  | 0     
 D2_45                                                  | 0     
 D2_60                                                  | 0     
 D2_90                                                  | 0     
 D3                                                     | 0     
 D4                                                     | 0     
 DEST_AIRPORT_ID                                        | 0     
 DEST_WBAN                                              | 0     
 FL_DATE                                                | 0     
 OP_CARRIER_AIRLINE_ID   

In [11]:
val cols = joinedData.columns.filter(_.toLowerCase.contains("destination_weather_relativehumidity"))
cols.foreach(println)

val selectedDf = joinedData.select(cols.map(col): _*)
selectedDf.show(10)

destination_weather_RelativeHumidity_h1
destination_weather_RelativeHumidity_h2
destination_weather_RelativeHumidity_h3
destination_weather_RelativeHumidity_Avg


cols = Array(destination_weather_RelativeHumidity_h1, destination_weather_RelativeHumidity_h2, destination_weather_RelativeHumidity_h3, destination_weather_RelativeHumidity_Avg)
selectedDf = [destination_weather_RelativeHumidity_h1: double, destination_weather_RelativeHumidity_h2: double ... 2 more fields]


+---------------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+
|destination_weather_RelativeHumidity_h1|destination_weather_RelativeHumidity_h2|destination_weather_RelativeHumidity_h3|destination_weather_RelativeHumidity_Avg|
+---------------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+
|                                   85.0|                                   77.0|                                   74.0|                       78.66666666666667|
|                                   43.0|                                   38.0|                                   35.0|                      38.666666666666664|
|                                   75.0|                                   72.0|                                   70.0|                       72.33333333333333|
|                     

[destination_weather_RelativeHumidity_h1: double, destination_weather_RelativeHumidity_h2: double ... 2 more fields]

In [12]:
val cols = joinedData.columns.filter(_.toLowerCase.contains("origin_weather_relativehumidity"))
cols.foreach(println)

val selectedDf = joinedData.select(cols.map(col): _*)
selectedDf.show(10)

origin_weather_RelativeHumidity_h1
origin_weather_RelativeHumidity_h2
origin_weather_RelativeHumidity_h3
origin_weather_RelativeHumidity_Avg
+----------------------------------+----------------------------------+----------------------------------+-----------------------------------+
|origin_weather_RelativeHumidity_h1|origin_weather_RelativeHumidity_h2|origin_weather_RelativeHumidity_h3|origin_weather_RelativeHumidity_Avg|
+----------------------------------+----------------------------------+----------------------------------+-----------------------------------+
|                              35.0|                              26.0|                              23.0|                               28.0|
|                              89.0|                              92.0|                              92.0|                               91.0|
|                              44.0|                              40.0|                              39.0|                               41.0|
|

cols = Array(origin_weather_RelativeHumidity_h1, origin_weather_RelativeHumidity_h2, origin_weather_RelativeHumidity_h3, origin_weather_RelativeHumidity_Avg)
selectedDf = [origin_weather_RelativeHumidity_h1: double, origin_weather_RelativeHumidity_h2: double ... 2 more fields]


[origin_weather_RelativeHumidity_h1: double, origin_weather_RelativeHumidity_h2: double ... 2 more fields]

In [13]:
val cols = joinedData.columns.filter(_.toLowerCase.contains("destination_weather_hourlyprecip"))
cols.foreach(println)

val selectedDf = joinedData.select(cols.map(col): _*)
selectedDf.show(10)

++
||
++
||
||
||
||
||
||
||
||
||
||
++
only showing top 10 rows



cols = Array()
selectedDf = []


[]

In [14]:
val cols = joinedData.columns.filter(_.toLowerCase.contains("origin_weather_hourlyprecip"))
cols.foreach(println)

val selectedDf = joinedData.select(cols.map(col): _*)
selectedDf.show(10)

cols = Array()
selectedDf = []


++
||
++
||
||
||
||
||
||
||
||
||
||
++
only showing top 10 rows



[]

In [15]:
val cols = joinedData.columns.filter(_.toLowerCase.contains("destination_weather_press_change_abs"))
cols.foreach(println)

val selectedDf = joinedData.select(cols.map(col): _*)
selectedDf.show(10)

destination_weather_press_change_abs_h1
destination_weather_press_change_abs_h2
destination_weather_press_change_abs_h3
destination_weather_press_change_abs_Max


cols = Array(destination_weather_press_change_abs_h1, destination_weather_press_change_abs_h2, destination_weather_press_change_abs_h3, destination_weather_press_change_abs_Max)
selectedDf = [destination_weather_press_change_abs_h1: double, destination_weather_press_change_abs_h2: double ... 2 more fields]


+---------------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+
|destination_weather_press_change_abs_h1|destination_weather_press_change_abs_h2|destination_weather_press_change_abs_h3|destination_weather_press_change_abs_Max|
+---------------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+
|                                   NULL|                                   NULL|                                   13.0|                                    13.0|
|                                   NULL|                                    3.0|                                   NULL|                                     3.0|
|                                   NULL|                                   NULL|                                   15.0|                                    15.0|
|                     

[destination_weather_press_change_abs_h1: double, destination_weather_press_change_abs_h2: double ... 2 more fields]

In [16]:
val cols = joinedData.columns.filter(_.toLowerCase.contains("origin_weather_press_change_abs"))
cols.foreach(println)

val selectedDf = joinedData.select(cols.map(col): _*)
selectedDf.show(10)

origin_weather_press_change_abs_h1
origin_weather_press_change_abs_h2
origin_weather_press_change_abs_h3
origin_weather_press_change_abs_Max


cols = Array(origin_weather_press_change_abs_h1, origin_weather_press_change_abs_h2, origin_weather_press_change_abs_h3, origin_weather_press_change_abs_Max)
selectedDf = [origin_weather_press_change_abs_h1: double, origin_weather_press_change_abs_h2: double ... 2 more fields]


+----------------------------------+----------------------------------+----------------------------------+-----------------------------------+
|origin_weather_press_change_abs_h1|origin_weather_press_change_abs_h2|origin_weather_press_change_abs_h3|origin_weather_press_change_abs_Max|
+----------------------------------+----------------------------------+----------------------------------+-----------------------------------+
|                               4.0|                              NULL|                              NULL|                                4.0|
|                              NULL|                              NULL|                              NULL|                               NULL|
|                              NULL|                              NULL|                               1.0|                                1.0|
|                              NULL|                              NULL|                              NULL|                               NULL|

[origin_weather_press_change_abs_h1: double, origin_weather_press_change_abs_h2: double ... 2 more fields]

In [17]:
val cols = joinedData.columns.filter(_.toLowerCase.contains("destination_weather_feature_temp_dewpoint_spread"))
cols.foreach(println)

val selectedDf = joinedData.select(cols.map(col): _*)
selectedDf.show(10)

destination_weather_feature_temp_dewpoint_spread_h1
destination_weather_feature_temp_dewpoint_spread_h2
destination_weather_feature_temp_dewpoint_spread_h3
destination_weather_feature_temp_dewpoint_spread_Min


cols = Array(destination_weather_feature_temp_dewpoint_spread_h1, destination_weather_feature_temp_dewpoint_spread_h2, destination_weather_feature_temp_dewpoint_spread_h3, destination_weather_feature_temp_dewpoint_spread_Min)
selectedDf = [destination_weather_feature_temp_dewpoint_spread_h1: double, destination_weather_feature_temp_dewpoint_spread_h2: double ... 2 more fields]


+---------------------------------------------------+---------------------------------------------------+---------------------------------------------------+----------------------------------------------------+
|destination_weather_feature_temp_dewpoint_spread_h1|destination_weather_feature_temp_dewpoint_spread_h2|destination_weather_feature_temp_dewpoint_spread_h3|destination_weather_feature_temp_dewpoint_spread_Min|
+---------------------------------------------------+---------------------------------------------------+---------------------------------------------------+----------------------------------------------------+
|                                 2.8000000000000007|                                  4.400000000000002|                                                5.0|                                  2.8000000000000007|
|                                 12.700000000000001|                                               15.0|                                               16.1

[destination_weather_feature_temp_dewpoint_spread_h1: double, destination_weather_feature_temp_dewpoint_spread_h2: double ... 2 more fields]

In [18]:
val cols = joinedData.columns.filter(_.toLowerCase.contains("origin_weather_feature_temp_dewpoint_spread"))
cols.foreach(println)

val selectedDf = joinedData.select(cols.map(col): _*)
selectedDf.show(10)

origin_weather_feature_temp_dewpoint_spread_h1
origin_weather_feature_temp_dewpoint_spread_h2
origin_weather_feature_temp_dewpoint_spread_h3
origin_weather_feature_temp_dewpoint_spread_Min
+----------------------------------------------+----------------------------------------------+----------------------------------------------+-----------------------------------------------+
|origin_weather_feature_temp_dewpoint_spread_h1|origin_weather_feature_temp_dewpoint_spread_h2|origin_weather_feature_temp_dewpoint_spread_h3|origin_weather_feature_temp_dewpoint_spread_Min|
+----------------------------------------------+----------------------------------------------+----------------------------------------------+-----------------------------------------------+
|                                          17.2|                                          21.6|                                          24.0|                                           17.2|
|                                           1.7

cols = Array(origin_weather_feature_temp_dewpoint_spread_h1, origin_weather_feature_temp_dewpoint_spread_h2, origin_weather_feature_temp_dewpoint_spread_h3, origin_weather_feature_temp_dewpoint_spread_Min)
selectedDf = [origin_weather_feature_temp_dewpoint_spread_h1: double, origin_weather_feature_temp_dewpoint_spread_h2: double ... 2 more fields]


[origin_weather_feature_temp_dewpoint_spread_h1: double, origin_weather_feature_temp_dewpoint_spread_h2: double ... 2 more fields]